In [2]:
import numpy as np
from typing import Callable
import abc

In [3]:
class Optimizer(abc.ABC):
    @abc.abstractmethod
    def __init__(self):
        pass
    @abc.abstractmethod
    def optimize(self):
        pass

In [4]:
class GradientDescentOptimizer(Optimizer):
    def __init__(self, learning_rate: float = 0.01):
        self.learning_rate = learning_rate
    
    def optimize(self, val, grad):
        return val - self.learning_rate * grad

In [5]:
class ActivationFunction(abc.ABC):
    @abc.abstractmethod
    def __call__(self, x):
        pass
    @abc.abstractmethod
    def gradient(self, x):
        pass

In [6]:
class Sigmoid(ActivationFunction):
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))
    
    def gradient(self, x):
        sig = self.__call(x)
        return sig * (1 - sig)

In [7]:
class ReLU(ActivationFunction):
    def __call__(self, x):
        return np.where(x >= 0, x, 0)
    
    def gradient(self, x):
        return np.where(x >= 0, 1, 0)

In [8]:
class Initializer(abc.ABC):
    @abc.abstractmethod
    def __call__(self, var):
        pass

In [9]:
class UniformInitializer(Initializer):
    def __call__(self, mean, var, shape):
        stddev = np.sqrt(var)
        lim = np.sqrt(3) * stddev
        
        return np.random.uniform(-lim, lim, shape)

In [10]:
class Layer(abc.ABC):
    @abc.abstractmethod
    def __init__(self):
        pass
    
    @abc.abstractmethod
    def set_optimizer(self, optimizer):
        pass
    
    @abc.abstractmethod
    def forward_pass(self, X):
        pass
    
    @abc.abstractmethod
    def backward_pass(self, X):
        pass
    
    @abc.abstractmethod
    def output_shape(self):
        pass

In [50]:
class Dense(Layer):
    def __init__(self, n_units, input_shape):
        self.n_units = n_units
        
        if len(input_shape) != 2:
            raise Exception('Input shape other than 2 not supported.')
        
        self.input_shape = input_shape
    
    def output_shape(self):
        return (self.n_units, self.input_shape[1])
    
    def set_activation(self, activation):
        if not isinstance(activation, Activation):
            raise Exception('The activation object provided is not an instance of Activation class.')
        
        self.activation = activation
    
    def initialize(self, initializer):
        if not isinstance(initializer, Initializer):
            raise Exception('The initializer object provided is not an instance of Initializer class.')
        
        self.initializer = initializer
        
        self.W = self.initializer(mean=0, var=1/self.n_units, shape=(self.n_units, self.input_shape[0]))
        self.b = self.initializer(mean=0, var=1/self.n_units, shape=(self.n_units, 1))
    
    def set_optimizer(self, optimizer):
        
        if not isinstance(optimizer, Optimizer):
            raise Exception('The optimizer object provided is not an instance of Optimizer class.')
        
        self.optimizer = optimizer
    
    def forward_pass(self, X):
        return self.W.dot(X) + b
    
    def backward_pass(self, cum_grad, X):
        grad_W = cum_grad.dot(X.T)
        grad_b = np.sum(cum_grad, axis=1)
        
        self.W = self.optimizer.optimize(self.W, grad_W)
        self.b = self.optimizer.optimize(self.b, grad_b)

In [58]:
dense_layer = Dense(n_units=5, input_shape=(3, 12))

In [59]:
dense_layer.output_shape()

(5, 12)

In [60]:
dense_layer.initialize(UniformInitializer())

In [61]:
dense_layer.set_optimizer(GradientDescentOptimizer())

In [62]:
dense_layer.W

array([[-0.71332766,  0.21712796, -0.63873653],
       [ 0.63485774,  0.46681522, -0.52448592],
       [ 0.17531463, -0.24592166, -0.26128886],
       [ 0.07211597, -0.61977781, -0.59363191],
       [ 0.47887525,  0.65701514, -0.74923248]])

In [63]:
dense_layer.backward_pass(cum_grad=np.ones((5, 12)), X=np.ones((3, 12)))

In [64]:
dense_layer.W

array([[-0.83332766,  0.09712796, -0.75873653],
       [ 0.51485774,  0.34681522, -0.64448592],
       [ 0.05531463, -0.36592166, -0.38128886],
       [-0.04788403, -0.73977781, -0.71363191],
       [ 0.35887525,  0.53701514, -0.86923248]])